<a href="https://colab.research.google.com/github/kdidi99/ml_heidelberg/blob/main/ex5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Exercise 5

Team members:

*   Rudolf Braun 
*   Jannik Schacht
*   Kieran Didi

# 5.1 Constructing the matrix X

In [ ]:
import numpy as np
import scipy
from scipy.sparse import coo_matrix

In [ ]:
def construct_X(M, alphas, Np=None):
    alphas = np.array(alphas)*np.pi/180
    if not Np: # set Np, if not already defined
        Np = int(M*2**0.5) 
    D = M**2
    N0 = len(alphas)
    N = N0*Np
    s0 = M/2 # offset to centralize the points
    
    Cgrid = np.mgrid[0:M,0:M] # grid to generate the indizes a_j and b_j
    C = np.array([Cgrid[1].flatten()-s0, Cgrid[0].flatten()-s0]) # indizes a_j , b_j centralized
    n = np.array([np.cos(alphas), np.sin(alphas)]) # normal vector of the sensor
    p = np.transpose(C).dot(n) # projection of the distance on the sensor
    
    # the position of the bin where the projection lies is calculated as float...
    # ... to get the next bin below (round down) and above (round up / round down and add 1)
    # the term np.mgrid[0:D,0:N0][1]*Np corresponds to N_p*i_0, so we get the index i = i_p + N_p*i_0
    Pos = np.transpose(p*Np/M/2**0.5+Np/2 + np.mgrid[0:D,0:N0][1]*Np).flatten()
    
    posdown = Pos.astype(int) # positions of the next lower bins
    weightsdown = 1-Pos%1 # the weight is 1-distance
    posup = posdown+1 # positions of the next higher bins
    weightsup = Pos%1 # the weights add up to 1
    
    i_indices = np.array([*posdown, *posup]) # indices of the sensor bins
    j_indices = np.array(list(range(D))*N0*2) # due to the construction of i_indices the j_indices become rather simple
    weights = np.array([*weightsdown, *weightsup]) # corresponding weights
    
    X = scipy.sparse.coo_matrix((weights, (i_indices, j_indices)), shape=(N, D), dtype = np.float32)
    return X

In [ ]:
X = construct_X(M = 10, alphas = [-33, 1, 42], Np = 15)
print(X.shape)